In [ ]:
from pathlib import Path

import evaluate
# import evaluate
from datasets import load_dataset, load_from_disk
from tqdm import tqdm
from transformers import (BertForQuestionAnswering, BertTokenizer,  # , Trainer
                          pipeline)

# import numpy as np
# import torch

In [2]:
model_name = "bert-base-uncased"
dataset_path = Path.cwd().parent / "data/tokenized_squad"

In [3]:
# Load model and tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
tokenized_dataset = load_from_disk(dataset_path)

In [ ]:
# Load model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Load SQuAD dataset
dataset = load_dataset("squad", split="validation")

# Initialize QA pipeline
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    device=-1,  # Use GPU if available (-1 for CPU)
)

# Prepare evaluation
squad_metric = evaluate.load("squad")
predictions = []
references = []

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [ ]:
pbar = tqdm(total=10570)

for sample in dataset:
    pbar.update(1)
    # Generate prediction
    prediction = qa_pipeline(
        question=sample["question"],
        context=sample["context"],
        max_seq_len=384,
        doc_stride=128,
        handle_impossible_answer=False,
    )

    # Format prediction
    formatted_prediction = {
        "id": sample["id"],
        "prediction_text": prediction["answer"],
    }  # nbqa E501

    # Format reference
    formatted_reference = {"id": sample["id"], "answers": sample["answers"]} 

    predictions.append(formatted_prediction)
    references.append(formatted_reference)

pbar.close()

  0%|          | 30/10570 [01:17<7:34:01,  2.58s/it]
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vladimir/Desktop/language-models-exploration/.venv/lib/python3.12/site-packages/transformers/data/__init__.py", line 15, in <module>
    from .data_collator import (
  File "/Users/vladimir/Desktop/language-models-exploration/.venv/lib/python3.12/site-packages/transformers/data/data_collator.py", line 25, in <module>
    from ..models.bert import BertTokenizer, BertTokenizerFast
  File "<frozen importlib._bootstra

KeyboardInterrupt: 

In [ ]:
results = squad_metric.compute(predictions=predictions, references=references)
print(f"F1 Score: {results['f1']:.2f}")
print(f"Exact Match: {results['exact_match']:.2f}")